Optimise CNOT in quantum system

In [5]:
#QuTiP
import numpy as np
import qutip.logging_utils as logging
logger = logging.get_logger()
#Local
import qsoconfig, qso, qsorun



Configuration

In [6]:
# Create the configuration object
# This holds the parameters for the pulse optimisation and any
# general parameters
cfg = qsoconfig.gen_optim_config(parse_cl_args=False)
# fid_type options pure_Choi_local|pure_Choi_global|unit_global
cfg.fid_type = 'pure_choi_local'
# stats_type options: standard|local
cfg.stats_type = 'local'
# If true then processing stats will be collected
cfg.gen_stats = True
# Print messaging level 0-3
cfg.verbosity = 0
# Pulse generator type.
# Options: 
# For descriptions see qutip.control.pulsegen.create_pulse_gen
# Only the random types make sense with multiple repetitions
cfg.p_type = 'RND'

# These will be used as amplitude bounds for the PulseGenerator and the Optimizer
cfg.amp_lbound = -np.Inf
cfg.amp_ubound = np.Inf
# Number of repetions for each pulse optimisation
cfg.num_reps = 10
# Number of cpus to utilise
cfg.num_cpus = 4

# Multi-processing options:-
#    <blank> - repetitions will be spread over available cpus
#    num_tslots - scenarios with different num_tslots spread over cpus
#    numer_acc_limit - automatic search numerical accuracy threshold
# Note that numer_acc_limit typically requires many repeated pulse optimisations.
# Meaning it will take a lot of cpu time.
cfg.mp_opt = ""

# Create the Optimizer object and its children
# (Dynamics, FidelityComputer, TerminationConditions...)
optim = qsoconfig.gen_optim_objects(cfg)


No parameter file. Using defaults in code.
evo_time=10.0
num_tslots=12


In [7]:
# Configure the quantum system
dyn = optim.dynamics
# Configure the quantum system
# Physical parameters of the model
# Any number of qubits >= 3 can choosen. However larger numbers take a lot of processing
dyn.num_qubits = 3
# interact options: Ising|Heisenberg| or combinations of xyz
dyn.interact = 'Ising'
# topology options: chain|star|full|ring
dyn.topology = 'chain'
# ctrls_type options: combinations of xyz
dyn.ctrls_type = 'XY'
# Interaction strengths (see qsoconfig.py for details)
dyn.coup_const = 1.0
# By CNOT gate is always on qubits 1&2
# These attribs can be used to permute the couplings to effectively move the gate qubits
# (see qsoconfig.py for details)
dyn.hspace_order = []
dyn.auto_hspace = False
dyn.hspace_01_sep = 0
dyn.hspace_0_idx = 0



In [8]:
# Pulse parameters
# Number of timeslots for the pulse discretisation
dyn.num_tslots = 12
# Time allowed for the gate to evolve
dyn.evo_time = 10.0

# Pulse optimisation termination conditions
# The optimisation algorithm will stop when any of these conditions are met
tc = optim.termination_conditions
# Target for the infidelity (1 - gate fidelity)
tc.fid_err_targ = 1e-3
# Sum of the gradients wrt optimisation parameters (timeslot amplitudes)
tc.min_gradient_norm = 1e-30
# Number of iterations of the algorithm
tc.max_iter = 2400
# Computation time (in seconds)
tc.max_wall_time = 120*60.0
# Relative change in fid_err between iterations
# (see qutip / scipy documentation for details)
tc.accuracy_factor = 1e5

# Fidelity computer
fid_comp = dyn.fid_computer

# This is the maximum precsion that sub-system fidelities are 'measured'
# see choi_closed_fidcomp.my_round for details
# Zero implies full machine precision
fid_comp.numer_acc = 0.0

# These next parameters are used in the automatic search for the numerical
# accuracy threshold
# If numer_acc_exact==false, then numer_acc, st_numer_acc, end_numer_acc
# will be treated as proportion of fid_err_targ (during config)
fid_comp.numer_acc_exact = False
fid_comp.st_numer_acc = 0.01
fid_comp.end_numer_acc = 0.2
# These proportions are used to determine the boundaries for the search
# They are proportions of the number number of successful repeats 
# for the scenario.
fid_comp.success_prop_uthresh = 0.95
fid_comp.success_prop_lthresh = 0.01

# Pulse generator
p_gen = optim.pulse_generator
# by default initial amplitudes will be between -1 and 1
# This can be changed by setting the bounds or scaling
p_gen.lbound = cfg.amp_lbound
p_gen.ubound = cfg.amp_ubound
p_gen.scaling = 1.0

# Create the targets and Hamiltonians

qsoconfig.config_dynamics(dyn)



Configuring drift...
... for 3 qubits
using Ising interactions


IndexError: list index out of range